In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

from shapely.geometry import Point
from shapely.geometry import LineString
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt

### Read the percolation results

In [2]:
perc_nodes =  pd.read_csv('nodes.csv')
perc_edges = pd.read_csv('edges.csv')

In [4]:
perc_edges['pair'] = perc_edges['Source'].astype('str') + '-' + perc_edges['Target'].astype('str')

### Read bike network

In [5]:
osm_bike = pd.read_csv('osm_bikepaths.csv')

In [6]:
osm_bike['pair'] = osm_bike['u'].astype('str') + '-' + osm_bike['v'].astype('str')

In [7]:
bike_links = osm_bike['pair']

### Filter the network

In [8]:
has_bike = perc_edges[perc_edges['pair'].isin(bike_links)]

In [9]:
has_bike['Weight'].describe()

count     8419.000000
mean      9730.965673
std       1128.247918
min       5000.000000
25%      10000.000000
50%      10000.000000
75%      10000.000000
max      10000.000000
Name: Weight, dtype: float64

In [10]:
no_bike = perc_edges[~perc_edges['pair'].isin(bike_links)]

In [11]:
no_bike['Weight'].describe()

count    9137.000000
mean     1176.406479
std       594.288858
min       521.000000
25%       716.000000
50%       996.000000
75%      1502.000000
max      4803.000000
Name: Weight, dtype: float64

In [12]:
len(has_bike)+len(no_bike),len(perc_edges)

(17556, 17556)

### Eliminate links that have a bike facility on them

In [2]:
no_bike.rename(columns={'Source':'source','Target':'target','Weight':'weight'},inplace=True)

In [1]:
perc_nodes.set_index('Id',inplace=True)

In [3]:
no_bike.set_index('source',inplace=True)
no_bike['source_lat'] = perc_nodes['lat']
no_bike['source_lon'] = perc_nodes['lon']
no_bike.reset_index(inplace=True)

In [4]:
no_bike.set_index('target',inplace=True)
no_bike['target_lat'] = perc_nodes['lat']
no_bike['target_lon'] = perc_nodes['lon']
no_bike.reset_index(inplace=True)

In [18]:
len(no_bike[no_bike['source_lat'].isnull()]),len(no_bike[no_bike['source_lon'].isnull()]),\
len(no_bike[no_bike['target_lat'].isnull()]),len(no_bike[no_bike['target_lon'].isnull()])

(0, 0, 0, 0)

In [19]:
crs_wgs='+init=epsg:4326'

In [20]:
crs_utm="+proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

In [21]:
nodes_source=no_bike[['source','source_lat','source_lon']]
nodes_source.rename(columns={'source':'node','source_lat':'lat','source_lon':'lon'},inplace=True)
nodes_target=no_bike[['target','target_lat','target_lon']]
nodes_target.rename(columns={'target':'node','target_lat':'lat','target_lon':'lon'},inplace=True)
nodes_latent=pd.concat([nodes_target,nodes_source])

In [22]:
nodes_latent.drop_duplicates('node',inplace=True)

In [23]:
geometry = [Point(xy) for xy in zip(nodes_latent.lon, nodes_latent.lat)]
node_latent_gdf = GeoDataFrame(nodes_latent, crs=crs_wgs, geometry=geometry)

In [24]:
# Change crs to calculate the buffer
node_latent_gdf=node_latent_gdf.to_crs(crs_utm)

In [25]:
node_latent_gdf.loc[:,'buffer']=node_latent_gdf['geometry'].buffer(50)

In [26]:
node_latent_gdf = node_latent_gdf.set_geometry('buffer')

In [27]:
node_latent_gdf=node_latent_gdf.to_crs(crs_wgs)

Open the projected cicloruta

In [29]:
new_cicloruta=gpd.read_file('./01-data/Shapefiles/projected_cicloruta.shp')
cicloruta=gpd.read_file('./01-data/Shapefiles/Red_Ciclorruta.shp')

In [33]:
new_cicloruta_proj=new_cicloruta.to_crs(crs_wgs)
cicloruta_proj=cicloruta.to_crs(crs_wgs)

In [5]:
intersects=gpd.sjoin(node_latent_gdf,new_cicloruta_proj[['geometry']],how='left',op='intersects')

In [36]:
intersects.rename(columns={'index_right':'has_new_cicloruta'},inplace=True)

In [38]:
intersects.drop_duplicates('node',inplace=True)

In [39]:
intersects_2=gpd.sjoin(intersects,cicloruta_proj[['geometry']],how='left',op='intersects')

In [40]:
intersects_2.drop_duplicates('node',inplace=True)

In [41]:
intersects_2.rename(columns={'index_right':'has_cicloruta'},inplace=True)

In [44]:
# Check
intersects_2.loc[:,'near_bike']=0
intersects_2.loc[intersects_2['has_new_cicloruta'].notnull(),'near_bike']=1
intersects_2.loc[intersects_2['has_cicloruta'].notnull(),'near_bike']=1

Check if a node from an edge of the percolation is near a cicloruta

In [46]:
source_near_bike=pd.merge(no_bike,intersects_2[['node','near_bike']],left_on='source',right_on='node',how='left')

In [47]:
source_near_bike.rename(columns={'near_bike':'source_near_bike'},inplace=True)

In [48]:
source_near_bike.drop('node',axis=1,inplace=True)

In [49]:
target_near_bike=pd.merge(source_near_bike,intersects_2[['node','near_bike']],left_on='target',right_on='node',how='left')

In [50]:
target_near_bike.drop('node',axis=1,inplace=True)
target_near_bike.rename(columns={'near_bike':'target_near_bike'},inplace=True)

In [52]:
target_near_bike.loc[:,'near_bike']=0
target_near_bike.loc[(target_near_bike['source_near_bike']==1)&(target_near_bike['target_near_bike']==1),'near_bike']=1

In [55]:
latent_shp=target_near_bike[['source','target','weight','source_lat','source_lon','target_lat','target_lon','near_bike']]

In [6]:
latent_shp.loc[:,'line']=latent_shp.apply(lambda row:([row['source_lon'],row['source_lat']],[row['target_lon'],\
                                                                                       row['target_lat']]),axis=1)

In [58]:
latent_shp.loc[:,'linestring']=latent_shp['line'].apply(lambda x: LineString(x))

In [60]:
no_bike_shp=latent_shp[latent_shp['near_bike']==0]

In [61]:
no_bike_shp.loc[:,'line']=no_bike_shp.apply(lambda row:([row['source_lon'],row['source_lat']],[row['target_lon'],\
                                                                                       row['target_lat']]),axis=1)

In [10]:
# Transform final into a geodataframe
no_bike_shp_gdf = GeoDataFrame(no_bike_shp, crs=crs_wgs, geometry='linestring')

In [66]:
no_bike_shp_gdf[['source','target','linestring','weight']].to_file('no_bike_latent_v17_giantF521.shp')